# 말로하는 지도 만들기라는 거창한 프로젝트를 위해... DB와의 통신이 필요해졌기에 
# Quickstart
- 이 가이드에서는 SQL 데이터베이스를 사용하여 Q&A 체인과 에이전트를 만드는 기본적인 방법을 살펴볼 것입니다. 
- 이 시스템들을 통해 SQL 데이터베이스에 대한 질문을 하고 자연어 답변을 받을 수 있습니다. 
- 두 시스템의 주요 차이점은 에이전트가 질문에 답하기 위해 필요한 만큼 데이터베이스를 반복적으로 쿼리할 수 있다는 것입니다.
# ⚠️ Security note ⚠️
- SQL 데이터베이스의 Q&A 시스템을 구축하는 것은 모델이 생성한 SQL 쿼리를 실행해야 합니다. 
- 이 과정에는 본질적인 위험이 따릅니다. 
- 데이터베이스 연결 권한을 항상 체인/에이전트의 필요에 맞게 가능한 한 좁게 설정해야 합니다. 
- 이는 위험을 완전히 제거하지는 않지만 줄일 수 있습니다. 일반적인 보안 모범 사례에 대해서는 여기를 참조하세요.
# Architecture
- 전반적으로, 모든 SQL 체인과 에이전트의 단계는 다음과 같습니다:
- 질문을 SQL 쿼리로 변환: 모델이 사용자 입력을 SQL 쿼리로 변환합니다.
- SQL 쿼리 실행: SQL 쿼리를 실행합니다.
- 질문에 답하기: 모델이 쿼리 결과를 사용하여 사용자 입력에 응답합니다.


In [1]:
from dotenv import load_dotenv
load_dotenv('../dot.env')
import os
import getpass


def _set_if_undefined(var: str):
    # 주어진 환경 변수가 설정되어 있지 않다면 사용자에게 입력을 요청하여 설정합니다.
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


# OPENAI_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("OPENAI_API_KEY")
# LANGCHAIN_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("LANGCHAIN_API_KEY")
# TAVILY_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("TAVILY_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "QA_SQL_CSV_Quickstart"



# 로컬에 설치한 DB와의 연결

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

# 

In [11]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# llm = ChatOpenAI(model="gpt-3.5-turbo",
#                 temperature=0,
#                 streaming=True,
#     callbacks=[StreamingStdOutCallbackHandler()]
#                 )
# llm = ChatOpenAI(
#     base_url="http://localhost:1234/v1", #-> lmstudio를 통해 열어놓은 서버로 llm을 구동하고 있는 상태, lmstudio에 서빙하고 있는 모델만 바꿔주면 새로 나온 모델을 시험해볼 수 있음
#     api_key="lm-studio",
#     model="mradermacher_f16/Joah-Remix-Llama-3-KoEn-8B-Reborn-GGUF",
#     temperature=0,
#     streaming=True,
#     callbacks=[StreamingStdOutCallbackHandler()]
# )
# llm = ChatOpenAI(
#     base_url = 'http://sionic.chat:8001/v1',
#     api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
#     model = 'xionic-ko-llama-3-70b',
#     temperature = 0.1,
#     streaming=True,
#     callbacks=[StreamingStdOutCallbackHandler()]
# )

llm = ChatOpenAI(
    base_url="http://localhost:1234/v1", #-> lmstudio를 통해 열어놓은 서버로 llm을 구동하고 있는 상태, lmstudio에 서빙하고 있는 모델만 바꿔주면 새로 나온 모델을 시험해볼 수 있음
    api_key="lm-studio",
    model="QuantFactory/dolphin-2.9-llama3-8b-GGUF",
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)


chain = create_sql_query_chain(llm, db) #-> db정보를 context로 주는 chain
response = chain.invoke({"question": "return sql query about How many employees are there"})
db.run(response)



SELECT COUNT(*) FROM "Employee";



'[(8,)]'